In [1]:
# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# preparing the data
data = pd.read_csv('fer2013.csv')
train = data.loc[data['Usage'] == 'Training']
test = data.loc[data['Usage'] == 'PrivateTest']

'''
Use print if the data is found to be missing
'''

'\nUse print if the data is found to be missing\n'

In [3]:
# prepare training data
%time
import warnings
warnings.filterwarnings("ignore")

width, height = 48, 48

train_datapoints = train['pixels'].tolist()

# getting features for training
X_train = []
for xseq in train_datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X_train.append(xx.astype('float32'))

X_train = np.asarray(X_train)
X_train = np.expand_dims(X_train, -1)

test_datapoints = test['pixels'].tolist()

# getting features for testing
X_test = []
for xseq in test_datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X_test.append(xx.astype('float32'))

X_test = np.asarray(X_test)
X_test = np.expand_dims(X_test, -1)

# getting labels for training
y_train = train['emotion']
y_train = np.asarray(y_train)

# getting labels for testing
y_test = test['emotion']
y_test = np.asarray(y_test)

print("Preprocessing Done")
print("Number of Training Samples: ", X_train.shape[0])
print("Number of Test Samples: ", X_test.shape[0])
print("Training Label Counts: ", np.bincount(y_train))
print("Training Data Shape: ", X_train.shape)
print("Training Label Shape: ", y_train.shape)
print("Training Label Counts: ", np.bincount(y_test))

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs
Preprocessing Done
Number of Training Samples:  28709
Number of Test Samples:  3589
Training Label Counts:  [3995  436 4097 7215 4830 3171 4965]
Training Data Shape:  (28709, 48, 48, 1)
Training Label Shape:  (28709,)
Training Label Counts:  [491  55 528 879 594 416 626]


In [4]:
# visualise training data

visualise = False

if visualise:

    # Mapping emotion label with expression
    labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    # Create a figure with 4 rows and 5 columns
    width = 10
    height = 10
    fig = plt.figure(figsize=(10, 15))
    columns = 4
    rows = 5

    import random

    # For each row and column, randomly select an image from the dataset and display it
    for i in range(1, columns*rows +1):
    #     img = np.random.randint(10, size=(h,w))
        fig.add_subplot(rows, columns, i)
        random_factor = random.randint(1,len(X_train))
        plt.imshow(X_train[random_factor].reshape(48,48), cmap = 'gray')
        plt.title(labels[y_train[random_factor]])
    plt.show()


In [5]:
# Normalise the data
X_train /= 255.0
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)
X_test /= 255.0
X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

import tensorflow as tf

# Convolutional layer 1
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
])

# Convolutional layer 2
model.add(tf.keras.layers.Conv2D(256, (5, 5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Dropout(0.25))

# Convolutional layer 3
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Dropout(0.25))

# Convolutional layer 4
model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(2, 2))
model.add(tf.keras.layers.Dropout(0.25))

# Compile the model
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 256)       819456    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 9, 9, 256)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 9, 9, 256)         0

In [6]:
# Train the model with the training=True flag.
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100


2023-08-24 17:23:30.528140: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


898/898 [==============================] - 401s 446ms/step - loss: 1.7023 - accuracy: 0.3076 - val_loss: 1.5260 - val_accuracy: 0.4032
Epoch 2/100
898/898 [==============================] - 411s 457ms/step - loss: 1.5013 - accuracy: 0.4121 - val_loss: 1.4014 - val_accuracy: 0.4609
Epoch 3/100
898/898 [==============================] - 413s 459ms/step - loss: 1.4200 - accuracy: 0.4492 - val_loss: 1.3395 - val_accuracy: 0.4806
Epoch 4/100
898/898 [==============================] - 414s 461ms/step - loss: 1.3729 - accuracy: 0.4686 - val_loss: 1.3020 - val_accuracy: 0.5032
Epoch 5/100
898/898 [==============================] - 414s 461ms/step - loss: 1.3317 - accuracy: 0.4889 - val_loss: 1.2610 - val_accuracy: 0.5230
Epoch 6/100
588/898 [==================>...........] - ETA: 2:20 - loss: 1.3050 - accuracy: 0.5041

In [ ]:
# Plot the accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Testing'], loc='upper left')
plt.savefig("initial_accuracy.png")
plt.show()

In [ ]:
# Save the model
model.save("initial_model.h5")